# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


In [5]:
query = '''create table rental_may (
rental_id int(11) unique not null,
rental_date datetime default null,
inventory_id mediumint unsigned default null,
customer_id smallint unsigned default null,
return_date datetime default null,
staff_id tinyint unsigned default null,
last_update timestamp default null,
constraint primary key (rental_id))'''

rental_may =pd.read_sql_query(query, engine)
rental_may

OperationalError: (pymysql.err.OperationalError) (1050, "Table 'rental_may' already exists")
[SQL: create table rental_may (
rental_id int(11) unique not null,
rental_date datetime default null,
inventory_id mediumint unsigned default null,
customer_id smallint unsigned default null,
return_date datetime default null,
staff_id tinyint unsigned default null,
last_update timestamp default null,
constraint primary key (rental_id))]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [7]:
query2 = ''' insert into rental_may
select * from rental
where rental_date like '2005-05-%' '''

rental_may =pd.read_sql_query(query2, engine)
rental_may

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [8]:
rental_may
# why do I get an error here? Why doesn't it show rental_may? I checked in sql and there was all the data I wanted

NameError: name 'rental_may' is not defined

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [9]:
query3 = '''create table rentals_june (
rental_id int(11) unique not null,
rental_date datetime default null,
inventory_id mediumint unsigned default null,
customer_id smallint unsigned default null,
return_date datetime default null,
staff_id tinyint unsigned default null,
last_update timestamp default null,
constraint primary key (rental_id))'''

rentals_june =pd.read_sql_query(query3, engine)
rentals_june

ResourceClosedError: This result object does not return rows. It has been closed automatically.

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [11]:
query4 = ''' insert into rentals_june
select * from rental
where rental_date like '2005-06-%' '''

rentals_june =pd.read_sql_query(query4, engine)
print(rental_june)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

## Check the number of rentals for each customer for May.


In [12]:
query5 = ''' select customer_id, count(rental_id) from rental_may
group by customer_id'''

a =pd.read_sql_query(query5, engine)
a

,customer_id,count(rental_id)
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5
...,...,...
515,191,2
516,351,1
517,10,1
518,136,1


## Check the number of rentals for each customer for June.
 

In [13]:
query6 = ''' select customer_id, count(rental_id) from rentals_june
group by customer_id'''

b =pd.read_sql_query(query6, engine)
b

,customer_id,count(rental_id)
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6
...,...,...
585,412,1
586,335,1
587,226,2
588,22,1


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [ ]:
# not sure whats the difference between the two above und this question?

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [14]:
c = a.merge(b, how='inner', on='customer_id')
c

,customer_id,count(rental_id)_x,count(rental_id)_y
0,130,2,6
1,459,1,7
2,408,3,3
3,333,1,4
4,222,5,2
...,...,...,...
507,431,1,7
508,191,2,6
509,351,1,3
510,10,1,5


In [15]:
c.columns =['customer_id', 'count_may', 'count_june']
c

,customer_id,count_may,count_june
0,130,2,6
1,459,1,7
2,408,3,3
3,333,1,4
4,222,5,2
...,...,...,...
507,431,1,7
508,191,2,6
509,351,1,3
510,10,1,5
